In [13]:
pip install SpeechRecognition
pip install python-bidi
pip install pyquran 
pip install PyAudio
pip install arabic-reshaper
pip install google-cloud-speech

In [1]:
import numpy
import nltk
from nltk import word_tokenize
import pyquran as q
from bidi.algorithm import get_display
import os

In [9]:
tr = q.quran.get_verse(1, 1, with_tashkeel=False)
tr

'بسم الله الرحمن الرحيم'

In [8]:
import speech_recognition as sr
import arabic_reshaper


# Record Audio
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Say something!")
    audio = r.listen(source)

# Speech recognition using Google Speech Recognition
try:
    text = r.recognize_google(audio, language='ar-AR')
    reshaped_text = arabic_reshaper.reshape(text)
    #reshaped_text.encode('utf8')
    print("You said: " + reshaped_text)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Google Speech Recognition service; {0}".format(e))

Say something!
result2:
{   'alternative': [   {   'confidence': 0.92682546,
                           'transcript': 'بسم الله الرحمن الرحيم'},
                       {'transcript': 'جسم الله الرحمن الرحيم'}],
    'final': True}
You said: ﺑﺴﻢ ﷲ ﺍﻟﺮﺣﻤﻦ ﺍﻟﺮﺣﻴﻢ


In [10]:
c = 'م'
d = 'ﻣ'
# print the ASCII value of assigned character in c
print("The ASCII value of '" + c + "' is", ord(c)) # Arabic Presentation Forms-A UFB50
print("The ASCII value of '" + d + "' is", ord(d)) # Arabic U0600

The ASCII value of 'م' is 1605
The ASCII value of 'ﻣ' is 65251


In [11]:
def levenshteinDistanceDP(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))
    
    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1
    
    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]


In [12]:
toknized_trg = word_tokenize(tr)
toknized_src = word_tokenize(reshaped_text)
print(toknized_trg)
print(toknized_src)
psrc = 0
ptrg = 0
itre = len(toknized_trg)
trget = []
while (psrc < itre and ptrg < itre):
    if (psrc == len(toknized_src)):
        trget.append("Missed")
        ptrg += 1
    elif(levenshteinDistanceDP(toknized_src[psrc] , toknized_trg[ptrg]) == 0):
        trget.append("Same")
        psrc += 1
        ptrg += 1
    else:
        diff = levenshteinDistanceDP(toknized_src[psrc] , toknized_trg[ptrg])
        if(len(toknized_trg[ptrg]) > len(toknized_src[psrc]) and diff <= len(toknized_trg[ptrg])/2):
            trget.append("Modefied")
            psrc += 1
            ptrg += 1
        elif(len(toknized_trg[ptrg]) <= len(toknized_src[psrc]) and diff <= len(toknized_src[psrc])/2): 
            trget.append("Modefied")
            psrc += 1
            ptrg += 1
        else:
            if(psrc != len(toknized_src)-1):
                diff = levenshteinDistanceDP(toknized_src[psrc+1] , toknized_trg[ptrg])
            if (diff != 0):
                trget.append("Missed")
                psrc += 1
                ptrg += 1
            else:
                trget.append("Same")
                psrc += 1
                ptrg += 1
                
print()
print(trget)

['بسم', 'الله', 'الرحمن', 'الرحيم']
['ﺑﺴﻢ', 'ﷲ', 'ﺍﻟﺮﺣﻤﻦ', 'ﺍﻟﺮﺣﻴﻢ']

['Missed', 'Missed', 'Missed', 'Missed']
